In [ ]:
import numpy as np
import pandas as pd
import mysql.connector


In [ ]:
#save key from keys.txt
with open('keys.txt') as f:
    lines = f.readlines()
    key = lines[0].strip()


In [ ]:
db = mysql.connector.connect(
    host="localhost",
    user="ramsus",
    passwd=key,
    database="pagos"
)


In [ ]:
mycursor = db.cursor()

mycursor.execute("DESCRIBE pagos")

In [ ]:
describe = mycursor.fetchall()

In [ ]:
for x in describe:
    print(x)

In [ ]:
excel = pd.read_csv('pagos.csv', encoding='latin-1', sep=';')
del excel['%']
excel.dropna(inplace=True)
# convert Si in True and No in False
excel['Nequi'] = excel['Nequi'].map({'Si': 1, 'No': 0})
excel['Datafono'] = excel['Datafono'].map({'Si': 1, 'No': 0})
excel['Transferencia'] = excel['Transferencia'].map({'Si': 1, 'No': 0})
excel['Efectivo'] = excel['Efectivo'].map({'Si': 1, 'No': 0})
# convert value to number and remove the $ symbol and .
excel['Valor'] = excel['Valor'].str.replace('$', '')
excel['Valor'] = excel['Valor'].str.replace('.', '')
excel['Valor'] = excel['Valor'].astype(int)
# convert date in format day/month/year to datetime
excel['Fecha'] = pd.to_datetime(excel['Fecha'], format='%d/%m/%Y')

excel.reset_index(drop=True, inplace=True)

excel

In [ ]:
# convert pandas to sql
for i in range(len(excel)):
      mycursor.execute(
            "INSERT INTO pagos (paciente, valor, actividad, fecha, Nequi, Datafono, Transferencia, Efectivo) \
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
            (excel['Paciente'][i], int(excel['Valor'][i]), excel['Actividad'][i], excel['Fecha'][i], int(excel['Nequi'][i]), int(excel['Datafono'][i]), int(excel['Transferencia'][i]), int(excel['Efectivo'][i]))
      )

In [ ]:
# check changes
db.commit()

# check changes
mycursor.execute("SELECT * FROM pagos")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)



In [ ]:
# close connection
db.close()